Perusahaan peminjaman sepeda di San Fransisco memintamu untuk menganalisa durasi perjalanan customer mereka. Untuk memudahkan analisa, lakukan hal-hal berikut ini:

## **STATISTIK DESKRIPTIF**

1. Lakukan perhitungan central tendency (mean, median, dan modus).
2. Lakukan pengecekan dispersi (standar deviasi, variance, dan range).
3. Lakukan pengecekan skewness dan kurtosis.
3. Analisa apakah ada data outlier berdasarkan sebaran datanya.
4. Cari tahu berapa persentase outlier terhadap data keseluruhan.
5. Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

## **API**

Buatlah API untuk:
- `Menampilkan` seluruh entry data setelah dilakukan handling outlier.
- `Menghapus` entry data, dari data yang telah dihandling outlier.

clue:
- Load data csv yang sudah diolah dengan pandas, kemudian konversi data ke dictionary `df.to_dict()` atau json `df.to_json()` untuk dapat diolah lebih lanjut dengan API menggunakan FastAPI.
- Pembuatan API dilakukan pada script `.py`
- Endpoint dan penamaan function pada API dibebaskan.


---



Data dapat diakses dari BigQuery dengan ketentuan:

- Project_id: `bigquery-public-data`
- Dataset: `san_francisco_bikeshare`
- Table: `bikeshare_trips`
- Ambil hanya kolom `duration_sec` saja
- Berikan LIMIT pada data yang diambil sebesar 3000 entry data

Koneksikan data di atas menggunakan `BigQuery` ke Google Colab dengan code berikut:
```py
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "rock-wonder-317907" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)
```

Untuk melakukan Query menggunakan cara ini, kamu dapat menggunakan method `client.query('Masukkan Querynya').to_dataframe()`. Outputnya akan berupa Pandas dataframe, sehingga harus import Pandas.

Contoh:

```py
df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at < "2022-07-01"
ORDER BY year,month ASC
''').to_dataframe()
```



In [ ]:
# Menghubungkan ke BigQuery
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Autentikasi
auth.authenticate_user()
print('Authenticated')

# Koneksi ke BigQuery
project_id = "hacktiv8lc2latihan"
client = bigquery.Client(project=project_id)

# Mengambil data dari BigQuery
query = '''
SELECT duration_sec
FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT 3000
'''

df = client.query(query).to_dataframe()

# Melihat data awal
print(df.head())


In [ ]:
import numpy as np
from scipy import stats

# Central Tendency (mean, median, modus)
mean_duration = df['duration_sec'].mean()
median_duration = df['duration_sec'].median()
mode_duration = stats.mode(df['duration_sec'])[0][0]

# Dispersi (standar deviasi, variance, range)
std_duration = df['duration_sec'].std()
variance_duration = df['duration_sec'].var()
range_duration = df['duration_sec'].max() - df['duration_sec'].min()

# Skewness dan Kurtosis
skewness_duration = df['duration_sec'].skew()
kurtosis_duration = df['duration_sec'].kurtosis()

# Print hasil analisis
print(f'Mean: {mean_duration}')
print(f'Median: {median_duration}')
print(f'Mode: {mode_duration}')
print(f'Standard Deviation: {std_duration}')
print(f'Variance: {variance_duration}')
print(f'Range: {range_duration}')
print(f'Skewness: {skewness_duration}')
print(f'Kurtosis: {kurtosis_duration}')


In [ ]:
# Deteksi Outlier menggunakan IQR
Q1 = df['duration_sec'].quantile(0.25)
Q3 = df['duration_sec'].quantile(0.75)
IQR = Q3 - Q1

# Batas atas dan bawah untuk outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Deteksi data outlier
outliers = df[(df['duration_sec'] < lower_bound) | (df['duration_sec'] > upper_bound)]
outliers_count = len(outliers)

# Persentase outlier
outliers_percentage = (outliers_count / len(df)) * 100

# Data tanpa outlier
df_clean = df[(df['duration_sec'] >= lower_bound) & (df['duration_sec'] <= upper_bound)]

# Perubahan pada mean dan median setelah outlier dihapus
new_mean_duration = df_clean['duration_sec'].mean()
new_median_duration = df_clean['duration_sec'].median()

print(f'Jumlah outlier: {outliers_count}')
print(f'Persentase outlier: {outliers_percentage}%')
print(f'Mean setelah outlier dihapus: {new_mean_duration}')
print(f'Median setelah outlier dihapus: {new_median_duration}')


In [ ]:
from fastapi import FastAPI
import pandas as pd

# Inisialisasi aplikasi FastAPI
app = FastAPI()

# Fungsi untuk load data
def load_data():
    # Ambil data dari file yang sudah dihandle outlier (contoh: csv)
    df_clean = pd.read_csv("clean_bikeshare_data.csv")
    return df_clean

# Endpoint untuk menampilkan semua data
@app.get("/get_data/")
def get_data():
    df_clean = load_data()
    return df_clean.to_dict(orient="records")

# Endpoint untuk menghapus data berdasarkan index
@app.delete("/delete_entry/{index}")
def delete_entry(index: int):
    df_clean = load_data()
    if index < len(df_clean):
        df_clean = df_clean.drop(index=index)
        # Save kembali setelah menghapus entry
        df_clean.to_csv("clean_bikeshare_data.csv", index=False)
        return {"message": "Entry deleted successfully."}
    else:
        return {"error": "Index out of range."}
